In [28]:
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from copy import deepcopy
from sklearn import model_selection
from keras.layers import Input, Dense, Activation, Reshape
from keras.models import Model, Sequential
from sklearn import covariance as covar
from sklearn import preprocessing
from sklearn import neighbors as knn

In [2]:
data = pd.read_csv("extendedData.csv").dropna()
data.reset_index(drop=True, inplace=True)
data = data.drop('Unnamed: 0', 1)
data = data.drop('address', 1)
data = data.drop('id', 1)
data = data.drop('name', 1)
data = data.drop('collection_id', 1)

In [3]:
for j, k in enumerate(data["DOB"]):
    data.at[j, "DOB"] = datetime.datetime.now().year - int(data.at[j, "DOB"][:4])
data.rename(columns={'DOB' : 'age'}, inplace=True)

In [4]:
le = preprocessing.LabelEncoder()

In [5]:
le.fit(data['city'])
data['city'] = le.transform(data['city'])
le.fit(data['sex'])
data['sex'] = le.transform(data['sex'])
le.fit(data['state'])
data['state'] = le.transform(data['state'])
le.fit(data['MARITAL_STATUS'])
data['MARITAL_STATUS'] = le.transform(data['MARITAL_STATUS'])
le.fit(data['TOBACCO'])
data['TOBACCO'] = le.transform(data['TOBACCO'])
data['PURCHASED'] = ["None" if type(i)==float else i for i in list(data["PURCHASED"])]
le.fit(data['PURCHASED'])
data['PURCHASED'] = le.transform(data['PURCHASED'])

In [37]:
data = data.apply(pd.to_numeric, errors='ignore')
data.columns #Predict Bronze and diff(bronze, silver)
data_processed = preprocessing.scale(data)

AttributeError: 'MinCovDet' object has no attribute 'location_'

In [35]:
model = knn.NearestNeighbors(n_neighbors=5)#, metric="mahalanobis", n_jobs=2)
train, test = model_selection.train_test_split(data_processed, test_size=0.2, random_state=42)
model.fit(train)

ValueError: Must provide either V or VI for Mahalanobis distance

In [8]:
# remaining_data = data[40000:] #random split, normalize data
# remaining_data.reset_index(drop=True, inplace=True)

In [9]:
# all_neighbors_bronze_prediction = np.asarray(all_neighbors_bronze_prediction)
# all_neighbors_bronze_prediction.reshape(35507, 10)

In [10]:
#closest = model.kneighbors(data.head(), 5)
all_disall_distancestances = model.kneighbors()[0].tolist()
all_neighbors = model.kneighbors()[1].tolist()
all_neighbors_bronze_labels = []
all_neighbors_diff_labels = []
all_neighbors_purchased_labels = []

In [11]:
for i, j in enumerate(all_neighbors):
    at_pos_bronze = []
    at_pos_diff = []
    at_pos_purchased = []
    for k in range(len(j)):
        bronze = data.at[all_neighbors[i][k], "BRONZE"]
        diff = data.at[all_neighbors[i][k], "SILVER"] - bronze
        purchased = data.at[all_neighbors[i][k], "PURCHASED"]
        at_pos_bronze.append(bronze)
        at_pos_bronze.append(diff)
        at_pos_bronze.append(purchased)
    all_neighbors_bronze_labels.append(at_pos_bronze)
    all_neighbors_diff_labels.append(at_pos_diff)
    all_neighbors_purchased_labels.append(at_pos_purchased)

In [12]:
len(all_neighbors_bronze_labels)

35507

In [40]:
model_nn = Sequential()
inputlayer = Dense(output_dim = 20, input_shape=(5,), activation='relu')
secondlayer = Dense(output_dim = 1, activation='sigmoid')
model_nn.add(inputlayer)
model_nn.add(secondlayer)

model_nn.compile(optimizer='rmsprop',
              loss='mean_squared_error', metrics=["accuracy"])
print(model_nn.summary())
model_nn.fit(np.asarray(all_distances), np.asarray(all_neighbors_bronze_labels), verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 20)                120       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 21        
Total params: 141
Trainable params: 141
Non-trainable params: 0
_________________________________________________________________
None


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_shape=(5,), activation="relu", units=20)`
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: Error when checking target: expected dense_6 to have shape (None, 1) but got array with shape (35507, 15)

# Prediction Loop

In [20]:
all_neighbors = model.kneighbors(test[1])
all_distances = all_neighbors[0].tolist()
all_distances

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


[[2.810578611410689,
  2.9625814786363907,
  2.9655602874886635,
  2.9997274025968084,
  3.0106079717366305]]

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.]], dtype=float32)